In [1]:
import arviz as az
import bambi as bmb
import numpy as np
import pandas as pd
import pymc as pm
import pytensor.tensor as pt

In [2]:
data = pd.read_csv("tests/data/inhaler.csv")
data["rating"] = pd.Categorical(data["rating"], categories=[1, 2, 3, 4])
data.head()

,subject,rating,treat,period,carry
0,1,1,0.5,0.5,0
1,2,1,0.5,0.5,0
2,3,1,0.5,0.5,0
3,4,1,0.5,0.5,0
4,5,1,0.5,0.5,0


In [3]:
with pm.Model() as model:
    threshold = pm.Normal(
        "threshold", 
        mu=(-1, 0, 1),
        transform=pm.distributions.transforms.univariate_ordered
    )

    treat = pm.Normal("treat")
    period = pm.Normal("period")
    carry = pm.Normal("carry")

    eta = (
        treat * data["treat"].to_numpy() 
        + period * data["period"].to_numpy() 
        + carry * data["carry"].to_numpy() 
    )

    # like brms
    eta_shifted = threshold - pt.shape_padright(eta)
    probabilities = pm.math.sigmoid(eta_shifted)

    ps = pt.concatenate(
        [
            pt.shape_padright(probabilities[..., 0]), 
            probabilities[..., 1:] - probabilities[..., :-1], 
            pt.shape_padright(1 - probabilities[..., -1])
        ], 
        axis=-1
    )

    pm.Categorical("response", p=ps, observed=data["rating"].cat.codes.to_numpy())
    idata = pm.sample()

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [threshold, treat, period, carry]


In [4]:
az.summary(idata, var_names=["threshold", "treat", "period", "carry"])

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
threshold[0],0.504,0.092,0.346,0.692,0.002,0.001,2300.0,1465.0,1.0
threshold[1],3.004,0.183,2.654,3.340,0.003,0.002,2829.0,1426.0,1.0
threshold[2],4.142,0.293,3.637,4.715,0.006,0.004,2174.0,1684.0,1.0
treat,-0.750,0.239,-1.158,-0.290,0.006,0.004,1594.0,1115.0,1.0
period,0.175,0.174,-0.156,0.513,0.004,0.003,2201.0,1559.0,1.0
carry,-0.232,0.169,-0.549,0.077,0.004,0.003,1725.0,1276.0,1.0


In [5]:
model = bmb.Model("rating ~ period + carry + treat", data, family="cumulative")
model

/home/tomas/Desktop/OSS/bambinos/bambi/bambi/formula.py:102: UserWarning: The intercept is omitted in ordinal families
  warnings.warn("The intercept is omitted in ordinal families")


       Formula: rating ~ period + carry + treat
        Family: cumulative
          Link: p = logit
  Observations: 572
        Priors: 
    target = p
        Common-level effects
            period ~ Normal(mu: 0.0, sigma: 5.0)
            carry ~ Normal(mu: 0.0, sigma: 3.5356)
            treat ~ Normal(mu: 0.0, sigma: 5.0)
        
        Auxiliary parameters
            rating_threshold ~ Normal(mu: [-2.  0.  2.], sigma: 1.0, transform: ordered)

In [6]:
idata = model.fit()

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [rating_threshold, period, carry, treat]


Sampling 2 chains for 1_000 tune and 1_000 draw iterations (2_000 + 2_000 draws total) took 10 seconds.
We recommend running at least 4 chains for robust computation of convergence diagnostics


In [7]:
az.summary(idata)

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
period,0.178,0.176,-0.161,0.491,0.004,0.003,2240.0,1480.0,1.0
carry,-0.214,0.171,-0.568,0.083,0.004,0.003,1980.0,1534.0,1.0
treat,-0.778,0.232,-1.232,-0.349,0.005,0.004,1985.0,1601.0,1.0
rating_threshold[0],0.501,0.088,0.344,0.673,0.002,0.001,2315.0,1418.0,1.0
rating_threshold[1],3.029,0.188,2.665,3.376,0.004,0.003,2344.0,1395.0,1.0
rating_threshold[2],4.229,0.316,3.661,4.843,0.007,0.005,2210.0,1569.0,1.0


**WORKS!!!**

---

pt.prod

In [93]:
with pm.Model() as model:
    threshold = pm.Normal("threshold", mu=(-1, 0, 1))

    treat = pm.Normal("treat")
    period = pm.Normal("period")
    carry = pm.Normal("carry")

    eta = (
        treat * data["treat"].to_numpy() 
        + period * data["period"].to_numpy() 
        + carry * data["carry"].to_numpy() 
    )

    eta_shifted = threshold - pt.shape_padright(eta)
    probabilities = pm.math.sigmoid(eta_shifted)
    
    n_columns = probabilities.type.shape[-1]
    ps = pt.concatenate(
        [
            pt.shape_padright(probabilities[..., 0]),
            *[
                pt.shape_padright(
                    probabilities[..., j] * pt.prod(1 - probabilities[..., :j], axis=-1)
                )
                for j in range(1, n_columns)
            ],
            pt.shape_padright(pt.prod(1 - probabilities, axis=-1))
        ],
        axis=-1
    )
    pm.Categorical("response", p=ps, observed=data["rating"].cat.codes.to_numpy())
    idata = pm.sample()

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [threshold, treat, period, carry]


Sampling 2 chains for 1_000 tune and 1_000 draw iterations (2_000 + 2_000 draws total) took 15 seconds.
We recommend running at least 4 chains for robust computation of convergence diagnostics


In [94]:
az.summary(idata)

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
threshold[0],0.538,0.091,0.378,0.718,0.002,0.001,2443.0,1170.0,1.0
threshold[1],2.166,0.207,1.780,2.569,0.004,0.003,2726.0,1354.0,1.0
threshold[2],1.091,0.397,0.345,1.848,0.008,0.006,2487.0,1431.0,1.0
treat,-0.709,0.217,-1.121,-0.311,0.004,0.003,2473.0,1610.0,1.0
period,0.206,0.164,-0.082,0.530,0.003,0.003,2677.0,1363.0,1.0
carry,-0.230,0.162,-0.511,0.095,0.003,0.003,2387.0,1534.0,1.0


```r
Population-Level Effects: 
             Estimate Est.Error l-95% CI u-95% CI Rhat Bulk_ESS Tail_ESS
Intercept[1]     0.55      0.09     0.38     0.73 1.00     5196     3204
Intercept[2]     2.26      0.22     1.84     2.71 1.00     5275     2661
Intercept[3]     1.09      0.43     0.27     1.98 1.00     5299     2562
treat           -0.75      0.22    -1.19    -0.31 1.00     3648     2724
period           0.21      0.17    -0.12     0.53 1.00     5490     3392
carry           -0.22      0.17    -0.53     0.11 1.00     3726     2606
```

**WORKS!**

---

In [ ]:
data = pd.read_csv("stemcell.csv")

In [ ]:
data.head()

,belief,rating,gender
0,fundamentalist,1,female
1,fundamentalist,1,female
2,fundamentalist,1,female
3,fundamentalist,1,female
4,fundamentalist,1,female


In [ ]:
data["rating"] = np.abs(data["rating"] - 5)
data["belief"] = pd.Categorical(data["belief"], categories=["moderate", "fundamentalist", "liberal"], ordered=True)

In [ ]:
np.unique(data["rating"])

array([1, 2, 3, 4])

In [ ]:
data.head()

,belief,rating,gender
0,fundamentalist,4,female
1,fundamentalist,4,female
2,fundamentalist,4,female
3,fundamentalist,4,female
4,fundamentalist,4,female


In [ ]:
belief_idx = data["belief"].cat.codes.to_numpy()
beliefs = ["moderate", "fundamentalist", "liberal"]

coords = {"beliefs": beliefs[1:]}
with pm.Model(coords=coords) as model:
    threshold = pm.Normal(
        "threshold", 
        mu=(-1, 0, 1),
        transform=pm.distributions.transforms.univariate_ordered
    )

    belief = pm.Normal("belief", dims="beliefs")
    belief = pt.concatenate([np.zeros(1), belief])

    eta = belief[belief_idx]

    # like brms
    eta_shifted = threshold - pt.shape_padright(eta)
    probabilities = pm.math.sigmoid(eta_shifted.T).T

    ps = pt.concatenate(
        [
            pt.shape_padright(probabilities[..., 0]), 
            probabilities[..., 1:] - probabilities[..., :-1], 
            pt.shape_padright(1 - probabilities[..., -1])
        ], 
        axis=-1
    )

    pm.Categorical("response", p=ps, observed=(data["rating"] - 1).to_numpy())
    idata = pm.sample()

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [threshold, belief]


Sampling 2 chains for 1_000 tune and 1_000 draw iterations (2_000 + 2_000 draws total) took 14 seconds.
We recommend running at least 4 chains for robust computation of convergence diagnostics


In [ ]:
az.summary(idata)

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
belief[fundamentalist],-0.387,0.154,-0.673,-0.088,0.004,0.003,1228.0,1160.0,1.0
belief[liberal],0.586,0.156,0.297,0.867,0.004,0.003,1498.0,1644.0,1.0
threshold[0],-2.124,0.143,-2.374,-1.833,0.004,0.003,1122.0,1278.0,1.0
threshold[1],-1.013,0.121,-1.246,-0.788,0.003,0.002,1225.0,1284.0,1.0
threshold[2],0.967,0.118,0.747,1.186,0.003,0.002,1355.0,1489.0,1.0


It matches what you get with brms if you do

```r
brm(
  formula = rating ~ 1 + belief,
  data = stemcell,
  family = cumulative("logit")
)
```

In [2]:
belief_idx = data["belief"].cat.codes.to_numpy()
beliefs = ["moderate", "fundamentalist", "liberal"]

coords = {"beliefs": beliefs[1:], "threshold_dim": [0, 1, 2]}

with pm.Model(coords=coords) as model:

    threshold = pm.Normal(
        "threshold", 
        mu=(-1, 0, 1),
        transform=pm.distributions.transforms.univariate_ordered,
        dims="threshold_dim"
    )

    belief = pm.Normal("belief", dims=("beliefs", "threshold_dim"))
    belief = pt.concatenate([pt.shape_padleft(pt.zeros_like(belief[0, ...])), belief])

    eta = belief[belief_idx]

    # # like brms
    eta_shifted = threshold - eta
    probabilities = pm.math.sigmoid(eta_shifted.T).T

    ps = pt.concatenate(
        [
            pt.shape_padright(probabilities[..., 0]), 
            probabilities[..., 1:] - probabilities[..., :-1], 
            pt.shape_padright(1 - probabilities[..., -1])
        ], 
        axis=-1
    )

    pm.Categorical("response", p=ps, observed=(data["rating"] - 1).to_numpy())
    #idata = pm.sample()

KeyError: 'belief'

In [55]:
az.summary(idata)

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
"belief[fundamentalist, 0]",-0.512,0.245,-0.970,-0.068,0.007,0.005,1173.0,1179.0,1.00
"belief[fundamentalist, 1]",-0.471,0.181,-0.787,-0.107,0.005,0.004,1246.0,1170.0,1.01
"belief[fundamentalist, 2]",-0.281,0.195,-0.636,0.088,0.005,0.004,1397.0,1292.0,1.00
"belief[liberal, 0]",0.173,0.286,-0.397,0.679,0.009,0.006,1132.0,1096.0,1.00
"belief[liberal, 1]",0.358,0.199,-0.013,0.719,0.005,0.004,1374.0,1298.0,1.00
"belief[liberal, 2]",0.735,0.176,0.419,1.065,0.005,0.004,1227.0,1424.0,1.00
threshold[0],-2.294,0.194,-2.663,-1.936,0.006,0.004,1021.0,881.0,1.00
threshold[1],-1.107,0.132,-1.382,-0.885,0.004,0.003,1383.0,1454.0,1.00
threshold[2],1.067,0.132,0.831,1.327,0.004,0.003,1268.0,1529.0,1.00


It matches what you get with brms if you do

```r
brm(
  formula = rating ~ 1 + cs(belief),
  data = stemcell,
  family = cumulative("logit")
)
```

* cumulative
* sratio
* acat

In [ ]:
def compute_cumulative_p(p, threhold):
    # cumulative model
    # P(Y = k) = F(tau_k - eta) - F(tau_{k - 1} - eta)
    ...

def compute_sratio_p():
    # sequential model
    # P(Y = k) = F(tau_k - eta) * prod_{j=1}^{k-1}{(1 - F(tau_j - eta))}
    ...

def compute_acat_p():
    # adjacent category model
    ...